# Recycling in Europe

This notebook processes and cleans multiple datasets related to recycling rates and environmental factors across European countries. The datasets are combined to create a unified dataset for analysis.

## Overview of Datasets:
- **D1**: General recycling rates (overall waste recycling %)
- **D2**: Recycling rates by waste type (glass, plastic, paper, metallic, wooden, packaging)
- **D3**: Socioeconomic indicators (GDP per capita, urbanization, internet usage, renewable energy, etc.)
- **CEI**: Circular economy indicators (private investments and output of circular economy sectors)
- **ENV**: Environmental tax revenues (as % of GDP)


# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from matplotlib.lines import Line2D
import statsmodels.api as sm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D

In [ ]:
MISSING_RATE = 0.9
EU_COUNTRIES = [
    'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 
    'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 
    'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 
    'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 
    'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 
    'Spain', 'Sweden'
]

# Preprocessing

## Functions

Generally, we used the following function to fill in missing values. The general logic is to use linear interpolation for intermediate gaps, followed by backard and forward filling to handle the edges.

In [ ]:
def fill_missing(series: pd.Series) -> pd.Series:
    # 1. interpolation for internal gaps
    # 2. backward fill for leading NaNs
    # 3. forward fill for trailing NaNs
    return series.interpolate().bfill().ffill()

## D1 - general recylcing rates

- **Why this dataset?**
    - We chose this dataset to serve as the primary benchmark for the analysis, providing high-level overview of how effectively different European nations manage their total waste output.

In [ ]:
d1 = pd.read_csv("raw_data/d1.csv")

In [ ]:
# copy
res_d1 = d1.copy()

# selecting the columns and renaming
res_d1 = res_d1[['geo', 'TIME_PERIOD', "OBS_VALUE", "OBS_FLAG"]]
res_d1 = res_d1.rename(columns={"geo": "country_name",
                                "TIME_PERIOD": "year",
                                "OBS_VALUE": "recycling_rate",
                                "OBS_FLAG": "flag"})

# types
res_d1['country_name'] = res_d1['country_name'].astype("string")
res_d1['year'] = res_d1['year'].astype(int)
res_d1['recycling_rate'] = res_d1['recycling_rate'].astype(float)
res_d1['flag'] = res_d1['flag'].astype("string")

# missing rates for later
missing_rates_d1 = pd.DataFrame(res_d1.groupby("country_name")["recycling_rate"].apply(lambda x: x.isna().mean())).reset_index()
missing_rates_d1.to_csv("processed_data/missing_value_rates_d1.csv", index=False)

# filtering based on missing rates
values_to_drop_d1 = []
for index, row in missing_rates_d1.iterrows():
    if row['recycling_rate'] > MISSING_RATE:
        values_to_drop_d1.append(row['country_name'])
res_d1 = res_d1[~res_d1['country_name'].isin(values_to_drop_d1)]
print(f"Dropped countries in d1 (n={len(values_to_drop_d1)}):", values_to_drop_d1)

# filling missing values
res_d1['recycling_rate_filled'] = res_d1.groupby("country_name")["recycling_rate"].transform(fill_missing)

In [ ]:
res_d1.isna().sum()

In [ ]:
res_d1.to_csv("processed_data/preprocessed_d1.csv", index=False)

## D2 - different types of waste

- Why this dataset?
    - This dataset was selected to provide necessary granularity, allowing us to identify if countries excel in processing specific materials. It helps reveal structural differences.

In [ ]:
d2 = pd.read_csv("raw_data/d2.csv")

In [ ]:
# copy
res_d2 = d2.copy()

# selecting the columns and renaming
res_d2 = res_d2[['geo', 'TIME_PERIOD', 'waste', 'OBS_VALUE', 'OBS_FLAG']]
res_d2 = res_d2.rename(columns={"geo": "country_name",
                                "TIME_PERIOD": "year",
                                "waste": "waste_type",
                                "OBS_VALUE": "recycling_rate",
                                "OBS_FLAG": "flag"})

# types
res_d2['country_name'] = res_d2['country_name'].astype("string")
res_d2['year'] = res_d2['year'].astype(int)
res_d2['waste_type'] = res_d2['waste_type'].astype("string")
res_d2['recycling_rate'] = res_d2['recycling_rate'].astype(float)
res_d2['flag'] = res_d2['flag'].astype("string")

# missing rates for later
missing_rates_d2 = pd.DataFrame(res_d2.groupby(["country_name", 'waste_type'])["recycling_rate"].apply(lambda x: x.isna().mean())).reset_index()
missing_rates_d2.to_csv("processed_data/missing_value_rates_d2.csv", index=False)

# filtering based on missing rates
values_to_drop_d2 = set()
for index, row in missing_rates_d2.iterrows():
    country_name = row['country_name']
    waste_type = row['waste_type']
    rate = row['recycling_rate']
    if rate > MISSING_RATE:
        values_to_drop_d2.add(country_name)
        print(f"{country_name}\t{waste_type}")
res_d2 = res_d2[~res_d2['country_name'].isin(values_to_drop_d2)]
print(f"Dropped countries in d2 (n={len(values_to_drop_d2)}):", values_to_drop_d2)

# filling missing values
res_d2['recycling_rate_filled'] = res_d2.groupby(["country_name", "waste_type"])["recycling_rate"].transform(fill_missing)

# pivoting waste types
clean_names = {
    "Glass packaging": "glass",
    "Metallic packaging": "metallic",
    "Packaging": "packaging",
    "Paper and cardboard packaging": "paper",
    "Plastic packaging": "plastic",
    "Wooden packaging": "wooden"
}
res_d2['waste_type'] = res_d2['waste_type'].map(clean_names)
res_d2 = res_d2.pivot(index=['country_name', 'year'], columns='waste_type', values=['recycling_rate_filled', "recycling_rate", "flag"]).reset_index()
res_d2.columns = [f"{val}_{waste}" if waste else val for val, waste in res_d2.columns]

In [ ]:
res_d2.isna().sum()

In [ ]:
res_d2.to_csv("processed_data/preprocessed_d2.csv", index=False)

## Creating overlapping countries set for D1 and D2 (both Eurostat)

We do this because D3 contains significantly more countries, and we would like to filter for these countries first there.

In [ ]:
for c in res_d1['country_name'].unique():
    if c not in res_d2['country_name'].unique():
        print(c)

In [ ]:
for c in res_d2['country_name'].unique():
    if c not in res_d1['country_name'].unique():
        print(c)

In [ ]:
values_to_drop_d1

In [ ]:
values_to_drop_d2

In [ ]:
countries = set(res_d1['country_name'].unique()).union(set(res_d2['country_name'].unique()))

## D3

- Why this dataset?
    - We included these World Development Indicators to investigate potential drivers of recycling performance, specifically whether economic wealth (GDP) and infrastructure density (urbanization) correlate with higher recylcing rates.

In [ ]:
d3 = pd.read_csv("raw_data/d3.csv")

Which countries are not present in D3?

In [ ]:
for c in countries:
    if c not in d3['Country Name'].unique():
        print(c)

In [ ]:
# manually searched for the differences
d3 = d3.replace("Slovak Republic", "Slovakia")
d3 = d3.replace("Kosovo", "Kosovo*")
d3 = d3.replace("Turkiye", "Türkiye")
for c in countries:
    if c not in d3['Country Name'].unique():
        print(c)

In [ ]:
# filtering for the countries in d1 and d2
d3 = d3[d3['Country Name'].isin(countries)]

In [ ]:
res_d3 = d3.copy()
res_d3 = res_d3.iloc[:-5, :] # last 5 rows are aggregates/metadata

# renaming columns
res_d3 = res_d3.rename(columns={"Time": "year",
                                "Country Name": "country_name",
                                "GDP per capita (constant 2015 US$) [NY.GDP.PCAP.KD]": "gdp_per_capita",
                                "Urban population (% of total population) [SP.URB.TOTL.IN.ZS]": "urban_population_pct",
                                "Individuals using the Internet (% of population) [IT.NET.USER.ZS]": "internet_users_pct",
                                "Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS]": "renewable_energy_pct",
                                "International tourism, number of arrivals [ST.INT.ARVL]": "tourism_arrivals",
                                "Central government debt, total (% of GDP) [GC.DOD.TOTL.GD.ZS]": "government_debt_pct_gdp",
                                "Population, total [SP.POP.TOTL]": "population_total",
                                "Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]": "manufacturing_value_added_pct_gdp",
                                "Government Effectiveness: Estimate [GE.EST]": "government_effectiveness_estimate",
                                "Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative) [SE.SEC.CUAT.UP.ZS]": "highschool_completed_pct",
                                "Households and NPISHs Final consumption expenditure per capita (constant 2015 US$) [NE.CON.PRVT.PC.KD]": "household_exp_percapita"})

# remove central government debt, too many missing values
res_d3 = res_d3[["country_name", "year", "gdp_per_capita", "urban_population_pct",
                 "internet_users_pct", "renewable_energy_pct", "tourism_arrivals",
                 "population_total", "manufacturing_value_added_pct_gdp",
                 "government_effectiveness_estimate", "highschool_completed_pct", "household_exp_percapita"]]

# types
res_d3 = res_d3.replace("..", np.nan)
res_d3['country_name'] = res_d3['country_name'].astype("string")
res_d3['year'] = res_d3['year'].astype(int)
res_d3['gdp_per_capita'] = res_d3['gdp_per_capita'].astype(float)
res_d3['urban_population_pct'] = res_d3['urban_population_pct'].astype(float)
res_d3['internet_users_pct'] = res_d3['internet_users_pct'].astype(float)
res_d3['renewable_energy_pct'] = res_d3['renewable_energy_pct'].astype(float)
res_d3['tourism_arrivals'] = res_d3['tourism_arrivals'].astype(float)
res_d3['population_total'] = res_d3['population_total'].astype(float)
res_d3['manufacturing_value_added_pct_gdp'] = res_d3['manufacturing_value_added_pct_gdp'].astype(float)
res_d3['government_effectiveness_estimate'] = res_d3['government_effectiveness_estimate'].astype(float)
res_d3['household_exp_percapita'] = res_d3['household_exp_percapita'].astype(float)
res_d3['highschool_completed_pct'] = res_d3['highschool_completed_pct'].astype(float)

# filtering based on missing rates
values_to_drop_d3 = []
missing_rates_d3 = pd.DataFrame()
missing_rates_d3['country_name'] = res_d3['country_name'].unique()
for col in res_d3.columns:
    if col in ["country_name", "year"]:
        continue
    missing_rates_d3_col = pd.DataFrame(res_d3.groupby("country_name")[col].apply(lambda x: x.isna().mean())).reset_index()
    missing_rates_d3 = pd.merge(missing_rates_d3, missing_rates_d3_col, on='country_name', how='left', suffixes=('', f'_{col}'))
    for index, row in missing_rates_d3_col.iterrows():
        if row[col] > MISSING_RATE:
            values_to_drop_d3.append(row['country_name'])
            print(f"{row['country_name']}\t{col}")
missing_rates_d3.to_csv("processed_data/missing_value_rates_d3.csv", index=False)
res_d3 = res_d3[~res_d3['country_name'].isin(values_to_drop_d3)]

# filling missing values
for col in res_d3.columns:
    if col in ["country_name", "year"]:
        continue
    res_d3[f"{col}_filled"] = res_d3.groupby("country_name")[col].transform(fill_missing)

In [ ]:
res_d3.isna().sum()

In [ ]:
res_d3.to_csv("processed_data/preprocessed_d3.csv", index=False)

## CEI - Private investment and gross value added (circular economy)

- Why this dataset?
    - This dataset was chosen to measure the economic commitment to sustainability, tracking the actual private investment and financial output of the circular economy sector.

In [ ]:
cei = pd.read_csv("raw_data/cei_cie012.csv")
cei

In [ ]:
res_cei = cei.copy()

# renaming countries
res_cei = res_cei.rename(columns={
    "geo": "country_name",
    "TIME_PERIOD": "year",
    "OBS_VALUE": "value",
    "indic_env": "indicator"
})

# filtering for relevant indicators, units, and countries
res_cei = res_cei[res_cei["unit"] == "Million euro"]
res_cei = res_cei[res_cei["country_name"].isin(EU_COUNTRIES)]
res_cei = res_cei[["country_name", "year", "indicator", "value"]]

# types
res_cei["country_name"] = res_cei["country_name"].astype("string")
res_cei["year"] = res_cei["year"].astype(int)
res_cei["indicator"] = res_cei["indicator"].astype("string")
res_cei["value"] = res_cei["value"].astype(float)

# renaming indicators
res_cei.loc[res_cei["indicator"] == "Gross value added", "indicator"] = "gross_val_add_mill"
res_cei.loc[res_cei["indicator"] == "Investment", "indicator"] = "priv_invest_mill"

print("Unique indicators:", res_cei["indicator"].unique())
print("\nData shape:", res_cei.shape)
res_cei

In [ ]:
# Compute missing rates and drop countries
values_to_drop_cei = []
missing_rates_cei = pd.DataFrame()

for indicator in res_cei["indicator"].unique():
    subset = res_cei[res_cei["indicator"] == indicator]
    mr = pd.DataFrame(subset.groupby("country_name")["value"].apply(lambda x: x.isna().mean())).reset_index()
    mr.columns = ["country_name", "missing_rate"]
    mr["indicator"] = indicator
    
    for _, row in mr.iterrows():
        if row["missing_rate"] > MISSING_RATE:
            values_to_drop_cei.append(row["country_name"])
            print(f"{row['country_name']}\t{indicator}")
    
    missing_rates_cei = pd.concat([missing_rates_cei, mr], ignore_index=True)

missing_rates_cei.to_csv("processed_data/missing_value_rates_cei.csv", index=False)
values_to_drop_cei = list(set(values_to_drop_cei))
print(f"\nDropped countries in CEI (n={len(values_to_drop_cei)}):", values_to_drop_cei)

res_cei = res_cei[~res_cei["country_name"].isin(values_to_drop_cei)]

print(f"\nRemaining countries: {res_cei['country_name'].nunique()}")
print(f"Remaining rows: {len(res_cei)}")
res_cei

In [ ]:
# Fill missing per (country, indicator) time series
res_cei['value_filled'] = res_cei.groupby(['country_name', 'indicator'])['value'].transform(fill_missing)

# Pivot to wide format: each indicator becomes a column
res_cei_wide = res_cei.pivot_table(
    index=["country_name", "year"],
    columns="indicator",
    values=['value', 'value_filled'],
    aggfunc="first"
).reset_index()

# Flatten multi-level columns
res_cei_wide.columns = [f"{ind}_{val}" if ind else val 
                        for val, ind in res_cei_wide.columns]
res_cei_wide.columns = res_cei_wide.columns.str.replace('_value', '')

print("Final shape:", res_cei_wide.shape)
print("\nColumns:", list(res_cei_wide.columns))
print("\nMissing values:")
print(res_cei_wide.isna().sum())

res_cei_wide.to_csv("processed_data/preprocessed_cei.csv", index=False)
print("\nSaved to processed_data/preprocessed_cei.csv")

## ENV - Environmental tax revenues

- Why this dataset?
    - We selected this dataset to analyse the role of fiscal policy, acting as a proxy for government intervention. It allows us to test whether higher environmental taxes successfully help nations to adopt better recycling practices.

In [ ]:
env = pd.read_csv("raw_data/env_ac_tax.csv")
env

In [ ]:
# ENV processing (long-format approach, then pivot)
res_env = env.copy()

# keep relevant columns
res_env = res_env[['geo', 'tax', 'TIME_PERIOD', 'OBS_VALUE']]
res_env = res_env.rename(columns={
    'geo': 'country_name',
    'TIME_PERIOD': 'year',
    'OBS_VALUE': 'value'
})

# types
res_env['country_name'] = res_env['country_name'].astype('string')
res_env['tax'] = res_env['tax'].astype('string')
res_env['year'] = res_env['year'].astype(int)
res_env['value'] = res_env['value'].astype(float)

res_env.loc[res_env["tax"] == "Total environmental taxes", "tax"] = "total_environm_tax_mill"
res_env.loc[res_env["tax"] == "Taxes on pollution/resources", "tax"] = "pollut_environm_tax_mill"

print("Unique indicators:", res_env["tax"].unique())
print("\nData shape:", res_env.shape)
res_env

In [ ]:
values_to_drop_env = []
missing_rates_env = pd.DataFrame()

for tax in res_env["tax"].unique():
    subset = res_env[res_env["tax"] == tax]
    mr = pd.DataFrame(subset.groupby("country_name")["value"].apply(lambda x: x.isna().mean())).reset_index()
    mr.columns = ["country_name", "missing_rate"]
    mr["tax"] = tax
    
    for _, row in mr.iterrows():
        if row["missing_rate"] > MISSING_RATE:
            values_to_drop_env.append(row["country_name"])
            print(f"To drop: {row['country_name']} due to {tax} ({row['missing_rate']:.2%})")
    
    missing_rates_env = pd.concat([missing_rates_env, mr], ignore_index=True)

missing_rates_env.to_csv("processed_data/missing_value_rates_env.csv", index=False)
values_to_drop_env = list(set(values_to_drop_env))
res_env = res_env[~res_env["country_name"].isin(values_to_drop_env)]

print(f"\nDropped countries in ENV (n={len(values_to_drop_env)}):", values_to_drop_env)
print(f"Remaining countries: {res_env['country_name'].nunique()}")

In [ ]:
res_env['value_filled'] = res_env.groupby(['country_name', 'tax'])['value'].transform(fill_missing)

res_env_wide = res_env.pivot_table(
    index=["country_name", "year"],
    columns="tax",
    values=['value', 'value_filled'],
    aggfunc="first"
).reset_index()

res_env_wide.columns = [f"{ind}_{val}" if ind else val 
                        for val, ind in res_env_wide.columns]

res_env_wide.columns = res_env_wide.columns.str.replace('_value', '')

print("\nFinal shape:", res_env_wide.shape)
print("Columns:", list(res_env_wide.columns))
print("\nMissing values after filling:")
print(res_env_wide.isna().sum())

res_env_wide.to_csv("processed_data/preprocessed_env.csv", index=False)
print("\nSaved to processed_data/preprocessed_env.csv")

## Joins

### D1 and D3

In [ ]:
res_d1_d3 = pd.merge(res_d1, res_d3, on=['country_name', 'year'], how='inner')

In [ ]:
res_d1_d3.isna().sum()

In [ ]:
for index, row in res_d1.iterrows():
    c = row['country_name']
    y = row['year']
    if res_d1_d3[(res_d1_d3['country_name'] == c) & (res_d1_d3['year'] == y)].empty:
        print(c, y)

These values were dropped before, we can move on and save the dataset. Countries from the intersection (see before) are all present.

In [ ]:
res_d1_d3.to_csv("processed_data/preprocessed_d1_d3.csv", index=False)

### D2 and D3

In [ ]:
res_d2_d3 = pd.merge(res_d2, res_d3, on=['country_name', 'year'], how='inner')

In [ ]:
res_d2_d3.isna().sum()

In [ ]:
for index, row in res_d2.iterrows():
    c = row['country_name']
    y = row['year']
    if res_d2_d3[(res_d2_d3['country_name'] == c) & (res_d2_d3['year'] == y)].empty:
        print(c, y)

There were dropped before, we can move on and save it.

In [ ]:
res_d2_d3.to_csv("processed_data/preprocessed_d2_d3.csv", index=False)

## All datasets together

In [ ]:
def clean_and_unify_cols(df):
    cols_to_drop = [c for c in df.columns if 'flag' in c]
    df = df.drop(columns=cols_to_drop)
    
    filled_cols = [c for c in df.columns if '_filled' in c]
    
    mapping = {}
    for col in filled_cols:
        clean_name = col.replace('_filled', '')
        if clean_name in df.columns:
            df = df.drop(columns=[clean_name])
        mapping[col] = clean_name
        
    df = df.rename(columns=mapping)
    return df

dfs = [res_d1, res_d2, res_d3, res_cei_wide, res_env_wide]

# Merge secuencial
df_all = clean_and_unify_cols(dfs[0])
for df in dfs[1:]:
    df_all = df_all.merge(clean_and_unify_cols(df), on=['country_name', 'year'], how='outer')

df_all = df_all[df_all['country_name'].isin(EU_COUNTRIES)].sort_values(['country_name', 'year'])

df_all['total_environm_tax_per_capita'] = (df_all['total_environm_tax_mill'] * 1e6) / df_all['population_total']
df_all = df_all.drop(columns=['total_environm_tax_mill'])

df_all['pollut_environm_tax_per_capita'] = (df_all['pollut_environm_tax_mill'] * 1e6) / df_all['population_total']
df_all = df_all.drop(columns=['pollut_environm_tax_mill'])

df_all['gr_val_add_per_capita'] = (df_all['gross_val_add_mill'] * 1e6) / df_all['population_total']
df_all = df_all.drop(columns=['gross_val_add_mill'])

df_all['priv_inv_per_capita'] = (df_all['priv_invest_mill'] * 1e6) / df_all['population_total']
df_all = df_all.drop(columns=['priv_invest_mill'])

# Compute year range using non-null years
years_non_null = df_all['year'].dropna().astype(int) if not df_all['year'].dropna().empty else pd.Series(dtype='int')
min_year = int(years_non_null.min()) if not years_non_null.empty else None
max_year = int(years_non_null.max()) if not years_non_null.empty else None

print(f"Final dataset shape: {df_all.shape}")
print(f"Countries: {df_all['country_name'].nunique()}")
print(f"Years: {min_year} - {max_year}")
print(f"\nMissing values:\n{df_all.isna().sum().sum()} total missing values")
print(f"\nColumns: {len(df_all.columns)}")
df_all.to_csv("processed_data/preprocessed_all.csv", index=False)
df_all

## Result of preprocessing

1. `res_d1`
    1. Dataframe about the basic recycling rates of European countries
    2. Time: 2000-2023
    3. Preprocessing steps:
        1. Load the raw data.
        2. Select the important columns and rename them.
        3. Set the correct types for the columns.
        4. Calculated and saved missing rates
            1. It can be found in `missing_rates_d1`.
        5. Dropped values based on missing rates.
            1. No countries were dropped.
        6. Filled missing values based on logic mentioned in the Functions section.
            1. Filled values can be found in the `recycling_rate_filled` column.
        7. Saved the preprocessed data.

2. `res_d2`
    1. Dataframe about the different recycling rates in European countries
    2. Time: 2012-2023
    3. Preprocessing steps:
        1. Load the raw data.
        2. Select the important columns and rename them.
        3. Set the correct types for the columns.
        4. Calculated and saved missing rates
            1. It can be found in `missing_rates_d2`.
        5. Dropped values based on missing rates.
            1. Had to drop several countries.
        6. Filled missing values based on logic mentioned in the Functions section.
            1. After pivotong (see step 7.): Filled values can be found in new column (`recycling_rate_filled_{glass, metallic, packaging, paper, plastic, wooden}`)
        7. Pivoted the data.
            1. Before, there were two columns: the waste type and the recyling rate for the corresponding waste type.
            2. Now, we have a column for each waste type.
        7. Saved the preprocessed data.
3. `res_d3`
    1. Dataframe about different indicators for European countries
    2. Filled values can be found in a similar way.
    3. Decided to drop government debt column, because more than 20 countries had it fully missing.
    4. Dropped Bulgaria (100% missing in manufacturing), dropped Kosovo (100% missing in renewable, and tourism)
    5. Saved missing rates into csv (also the variable: `missing_rates_d3`), be aware (although better than D1)
        1. We can later change it to drop a country if it has missing values (or over a %).
    6. Time: 2000-2024
    3. Preprocessing steps:
        1. Load the raw data.
        2. Filter for countries that were both present in D1 and D2.
            1. We do this because D3 contains significantly more countries, and it's easier to start the preprocessing with less countries.
            2. Some countries were not found in D3.
                1. Manually checked the raw files, these countries used different names, we had to change these.
        3. Drop last 5 rows, because it is aggregate/metadata.
        4. Rename the columns.
        5. Set the correct types for the columns.
        6. Calculated and saved missing rates.
            1. It can be found in `missing_rates_d3`.
        7. Dropped values based on missing rates.
            1. Dropped Bulgaria (100% missing in manufacturing), dropped Kosovo (100% missing in renewable, and tourism)
            2. Decided to drop the government debt column, because more than 20 countries had it fully missing.
        8. Filled missing values based on logic mentioned in the Functions section.
        9. Saved the preprocessed data.
4. Circular Economy Indicators (CEI)
    1. Description* This dataset tracks the economic performance of circular economy sectors (recycling, repair, reuse) across European countries. It measures:
        1. Private investments: Gross investments in tangible goods.
        2. Gross value added: Economic output of circular economy sectors.
    2. Unit: Millions of euros. Will join with population to get Mill/pop.
    4. Coverage: All EU Member States, time series from 2005 onwards
    5. Preprocessing steps:
        1. Filter for investments as Millions of euros.
        2. Drop countries with missing values
        3. Pivot wide format (indicators as columns)
        4. Save cleaned dataset
    6. Results: Found no missing values in the dataframe. The resulting dataframe contains valid datapoints for all EU Member States.
5. Environmental Tax Revenues
    1. Description: This dataset contains government tax revenues from environmental taxes across European countries. Environmental taxes target specific activities with proven negative environmental impact. The dataset contains also the specific taxes on Pollution & Resources. This taxes on emissions (air, water), waste management, water abstraction, and raw material extraction.
    2. Unit: Millions of euros. Will join with population to get Mill/pop.
    3. Coverage: All EU Member States, Iceland, Norway, Switzerland; time series from 1995 onwards
    4. Preprocessing steps:
        1. Select relevant columns (country, year, tax revenue)
        2. Drop countries with missing values
        3. Pivot wide format (tax as columns)
        4. Save cleaned dataset
    5. Result: Found missing values for both taxes, the environmental and the specific in pollution and resources, on Liechtenstein and Switzerland. Dropped values of Liechtenstein as there were 100% missing values for `pollut_environm_tax_mill`.
6. Joins
    1. D1 and D3 -> `res_d1_d3`
    2. D2 and D3 -> `res_d2_d3`
    3. All five -> `df_all`

# Visualizations

The following part contains the visualisations relevant to the preprocessing. The first two graphs investigate the data in detail, they are not meant to be presented as they are quite crowded and contain a lot of elements. They are necessary however, to obtain understanding of the data on the country level by each variable.

First, we visualize the data of `res_d1_d3` (recycling rate) before and after imputation to uncover any trends in the data. As expected, we see that the missing data is on the edges, and is extrapolated with straight lines as a result of the fill. For the purpose of the analysis, we chose only EU countries as they are the most relevant. These countries do not have a high proportion of missing values.

The second graph shows each variable in more detail. We look at the already filled data and do not notice any concerning patterns.

In [ ]:
def plot_country_dashboard(df, primary_metrics, secondary_metric=None, 
                           cols=4, title="Country Analysis"):
    """
    Creates a grid of subplots for countries with support for 
    multiple primary metrics and an optional secondary Y-axis.
    """
    countries = df['country_name'].unique()
    n_countries = len(countries)
    rows = (n_countries // cols) + (1 if n_countries % cols > 0 else 0)

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), 
                             sharex=True, sharey=True)
    axes = axes.flatten()

    for i, country in enumerate(countries):
        ax1 = axes[i]
        data = df[df['country_name'] == country].sort_values('year')
        
        # Primary metrics, shared left axis
        for metric, style in primary_metrics.items():
            sns.lineplot(data=data, x='year', y=metric, ax=ax1, 
                         color=style.get('color'), 
                         linestyle=style.get('ls', '-'), 
                         linewidth=style.get('lw', 2), 
                         label=style.get('label', metric), legend=False)
        
        ax1.set_title(country, fontweight='bold', fontsize=14)
        ax1.set_ylim(-2, 100) # Assuming percentage/scaled data
        
        # Optional secondary metrics, right axis
        if secondary_metric:
            ax2 = ax1.twinx()
            sns.lineplot(data=data, x='year', y=secondary_metric, ax=ax2, 
                         color='purple', alpha=0.5, lw=1.5, legend=False)
            
            # Formatting right axis labels
            if i % cols != cols - 1:
                ax2.set_yticklabels([])
            else:
                ax2.set_ylabel(secondary_metric.replace('_', ' ').title())

    # Clean up empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Universal Legend
    handles1, labels1 = ax1.get_legend_handles_labels()
    fig.legend(handles1, labels1, loc='upper center', bbox_to_anchor=(0.5, 1.02), 
               ncol=min(len(labels1), 5), fontsize=12, frameon=False)

    plt.tight_layout()
    plt.suptitle(title, fontsize=20, y=1.05, fontweight='bold')
    return fig

In [ ]:
# Original vs imputed recycling rates
original_v_imputed = {
    'recycling_rate_filled': {'color': 'orange', 'lw': 2, 'alpha': 0.6, 'label': 'Imputed'},
    'recycling_rate': {'color': 'teal', 'lw': 1.5, 'marker': 'o', 'ms': 4, 'label': 'Original'}
}

plot_country_dashboard(res_d1_d3, original_v_imputed, cols=5);

In [ ]:
# Rescaling government effectiveness to 0-100 for better visualization since it is between -2.5 and 2.5
plot_df = res_d1_d3.copy()
plot_df['gov_effectiveness_index'] = ((plot_df['government_effectiveness_estimate_filled'] + 2.5) / 5 * 100)

# Primary metrics
metrics_config = {
    'recycling_rate_filled': {'color': 'orange', 'lw': 3, 'label': 'Recycling %'},
    'urban_population_pct_filled': {'color': 'blue', 'ls': '--', 'label': 'Urban Pop %'},
    'government_effectiveness_estimate_filled': {'color': 'black', 'ls': ':', 'label': 'Gov Index'},
    'renewable_energy_pct_filled': {'color': 'green', 'ls': '-.', 'label': 'Renewable Energy %'},
    'gdp_per_capita_filled': {'color': 'purple', 'ls': '-', 'label': 'GDP per Capita'},
    'manufacturing_value_added_pct_gdp_filled': {'color': 'brown', 'ls': '-', 'label': 'Manufacturing % GDP'},
    'population_total_filled': {'color': 'gray', 'ls': '-', 'label': 'Total Population'},
    'tourism_arrivals_filled': {'color': 'cyan', 'ls': '-', 'label': 'Tourism Arrivals'},
    'internet_users_pct_filled': {'color': 'magenta', 'ls': '--', 'label': 'Internet Users %'},
    'household_exp_percapita_filled': {'color': 'red', 'ls': '-', 'label': 'Household Exp per Capita'}
}

plot_country_dashboard(plot_df, metrics_config, secondary_metric='gdp_per_capita_filled');

The following graphs are the first ones to be included in the management summary. Because the data is so high-dimensional: many countries, years and variables, including all of it in a visualization makes it difficult to follow. Hence, we chose 5 countries in order to represent the case in different parts of the EU. We also plot the 2030 EU Recycling target to see how these countries compare against it.

All countries, except Sweden, show a steady increase in the recycling rates with Germany having met the target for several years now. In fact, Sweden even shows a decline in the recycling rates starting from 2020, possibly due to Covid-19. All countries show a plateau/slight decline in the years following Covid-19. This is likely due to the fact that the pandemic required a large number of resources, that had to be diverted elsewhere.

In [ ]:
# Selecting 5 target countries for the management summary
targets = ['Germany', 'Sweden', 'Italy', 'Czechia', 'Greece']
df_subset = res_d1_d3[res_d1_d3['country_name'].isin(targets)].copy()

sns.set_theme(style="whitegrid", rc={"axes.facecolor": "#f9f9f9"})
plt.figure(figsize=(12, 7))

palette = sns.color_palette("husl", len(targets))

line_plot = sns.lineplot(
    data=df_subset, 
    x='year', 
    y='recycling_rate_filled', 
    hue='country_name', 
    linewidth=3.5,
    marker='o',
    markersize=8,
    markeredgecolor='white',
    palette=palette
)

# EU 2030 target line
plt.axhline(y=60, color='#d62728', linestyle='--', linewidth=2, alpha=0.8)

plt.text(2000.5, 61, 'EU 2030 Target (60%)', color='#d62728', 
         fontweight='bold', fontsize=12, va='bottom')

plt.title("Path to 2030: Recycling Performance Tracking", 
          fontsize=20, fontweight='bold', pad=25, loc='left')


plt.ylabel("Recycling Rate (%)", fontsize=12, fontweight='bold')
plt.xlabel("Reporting Year", fontsize=12, fontweight='bold')

plt.legend(title="Target Countries", title_fontsize='13', 
           fontsize='11', bbox_to_anchor=(1.02, 1), loc='upper left', frameon=False)

sns.despine()

plt.tight_layout()

plt.savefig("../recycling-in-europe/figures/recycling_performance_2030.png", dpi=300, bbox_inches="tight")

plt.show()

Then, we move on to correlation analysis between the variables. This is important for several reasons, first it gives a preliminary overview which variables might be more important in our analysis. Second, it gives an overview of multicollinearity in the data that is important for modelling and regularization.

The graph shows that variables such as government effectiveness and percentage of internet users are strong drivers of recycling rates. Surprisingly, the percentage of renewable energy being utilized does not show a high correlation coefficient. Of course, it is important to not that correlation does not imply causation, and that this is just the preliminary step to obtain an overview.

In [ ]:
# Correlation analysis to identify drivers of recycling rates
core_indicators = [
    'recycling_rate_filled',
    'gdp_per_capita_filled',
    'urban_population_pct_filled',
    'renewable_energy_pct_filled',
    'government_effectiveness_estimate_filled',
    'manufacturing_value_added_pct_gdp_filled',
    'population_total_filled',
    'tourism_arrivals_filled',
    'internet_users_pct_filled',
    'household_exp_percapita_filled']

sns.set_theme(style="white")
plt.figure(figsize=(10, 8))

df_corr = res_d1_d3[core_indicators].corr()
df_corr.columns = [c.replace('_filled', '').replace('_', ' ').title() for c in df_corr.columns]
df_corr.index = [i.replace('_filled', '').replace('_', ' ').title() for i in df_corr.index]


mask = np.triu(np.ones_like(df_corr, dtype=bool))

sns.heatmap(
    df_corr, 
    mask=mask, 
    annot=True, 
    fmt=".2f", 
    cmap='RdYlGn', 
    center=0,
    square=True, 
    linewidths=.5, 
    cbar_kws={"shrink": .7, "label": "Correlation Coefficient ($r$)"},
    annot_kws={"size": 11, "weight": "bold"}
)

plt.title("What Drives Recycling?\nRelationship Analysis for Selected Countries", 
          fontsize=18, pad=25, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, fontsize=12)

plt.tight_layout()

plt.savefig("../recycling-in-europe/figures/recycling_drivers_correlation.png", dpi=300, bbox_inches="tight")

plt.show()

Now, we move on to more specific recycling data that is broken down by material profile: wood, plastic, paper, packaging, metallic, glass. One of the most dramatic elements to be noticed immediately is that the recycling rate of plastic for Sweden went down dramatically to 28.6%. This is important in order to understand exactly which parts of the recycling industry have imporoved and where.

In [ ]:
# Selecting the same 5 target countries for horizontal comparison
targets = ['Germany', 'Sweden', 'Italy', 'Czechia', 'Greece']
latest_year = res_d2_d3['year'].max()
material_cols = [c for c in res_d2_d3.columns if 'recycling_rate_filled_' in c]
material_labels = [m.replace('recycling_rate_filled_', '').title() for m in material_cols]

fig, axes = plt.subplots(len(targets), 1, figsize=(12, len(targets) * 3), sharex=True)
sns.set_theme(style="whitegrid")

for i, country in enumerate(targets):
    ax = axes[i]

    # extract data points
    d2013 = res_d2_d3[
        (res_d2_d3['country_name'] == country) & (res_d2_d3['year'] == 2013)
    ][material_cols].values.flatten()

    dLatest = res_d2_d3[
        (res_d2_d3['country_name'] == country) & (res_d2_d3['year'] == latest_year)
    ][material_cols].values.flatten()

    y_pos = np.arange(len(material_labels))

    # 2013 baseline
    ax.barh(y_pos, d2013, color='#e0e0e0', label='2013 Rate', height=0.6)

    # arrows for change
    for j, (start, end) in enumerate(zip(d2013, dLatest)):
        color = '#2ca02c' if end >= start else '#d62728'

        ax.annotate(
            '',
            xy=(end, j),
            xytext=(start, j),
            arrowprops=dict(
                arrowstyle='->',
                color=color,
                lw=3
            )
        )

    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(material_labels, fontweight='bold', fontsize=11)
    ax.set_title(
        f"Recycling Material Profile: {country}",
        fontsize=15,
        fontweight='bold',
        loc='left',
        pad=10
    )
    ax.set_xlim(-5, 105)

    # value labels
    for j, (start, end) in enumerate(zip(d2013, dLatest)):
        if end >= start:
            # improvement → label on the right
            x = end + 1
            ha = 'left'
        else:
            # decline → label on the left
            x = end - 1
            ha = 'right'

        ax.text(
            x, j,
            f"{end:.1f}%",
            va='center',
            ha=ha,
            fontsize=10,
            fontweight='bold'
        )


    sns.despine(left=True, bottom=True)


legend_elements = [
    Line2D([0], [0], color='#e0e0e0', lw=10, label='2013 Rate'),
    Line2D([0], [0], color='#2ca02c', lw=3, label='Improvement'),
    Line2D([0], [0], color='#d62728', lw=3, label='Decline')
]

fig.legend(
    handles=legend_elements,
    loc='upper center',
    bbox_to_anchor=(0.5, 1.02),
    ncol=3,
    frameon=False,
    fontsize=12
)

plt.tight_layout()

plt.savefig("../recycling-in-europe/figures/recycling_material_profile_comparison.png", dpi=300, bbox_inches="tight")

plt.show()


# Statistical Analysis

### Data Aggregation & Selection

Data averaged over **2019–2023** to mitigate annual volatility. Luxembourg is excluded due to statistical leverage (outlier GDP).

**Variable Classification:**
* **Targets:** Dependent variables (Recycling rates).
* **Policy Drivers:** Modifiable independent variables (Taxes, Investment, Governance).
* **Context:** Structural/Control variables (GDP, Education, Urbanization).

In [ ]:
targets = [
    'recycling_rate',
    'recycling_rate_glass',
    'recycling_rate_metallic',
    'recycling_rate_packaging',
    'recycling_rate_paper',
    'recycling_rate_plastic',
    'recycling_rate_wooden'
]
policy_drivers = [
    'total_environm_tax_per_capita',
    'pollut_environm_tax_per_capita',
    'priv_inv_per_capita',
    'government_effectiveness_estimate'
]
context_vars = [
    'gdp_per_capita',
    'gr_val_add_per_capita',
    'urban_population_pct',
    'renewable_energy_pct',
    'highschool_completed_pct',
    'household_exp_percapita'
]

start_year = 2019
end_year = 2023

df_all = pd.read_csv('processed_data/preprocessed_all.csv')
df_recent = df_all[(df_all['year'] >= start_year) & (df_all['year'] <= end_year)].copy()

all_vars = targets + policy_drivers + context_vars
existing_cols = [c for c in all_vars if c in df_recent.columns]

df_avg = df_recent.groupby('country_name')[existing_cols].mean().reset_index()
df_avg = df_avg[df_avg['country_name'] != 'Luxembourg']
regression_cols = ['recycling_rate'] + policy_drivers
df_clean_regression = df_avg.dropna(subset=regression_cols).copy()

### Unsupervised Learning: Country Clustering

**K-Means clustering** stratifies countries based on economic power (`gdp_per_capita`), outcome (`recycling_rate`), and institutional capacity (`government_effectiveness_estimate`).

**Objective:**
Identify structural heterogeneity. The scatter plot reveals performance tiers relative to wealth, isolating "overachievers" from resource-rich underperformers.

In [ ]:
features_cluster = ['gdp_per_capita', 'recycling_rate', 'government_effectiveness_estimate']
X_cluster = df_clean_regression[features_cluster].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_clean_regression['cluster'] = kmeans.fit_predict(X_scaled)

# Visualize clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=df_clean_regression,
    x='gdp_per_capita',
    y='recycling_rate',
    hue='cluster',
    palette='Set2',
    s=200,
    alpha=0.7,
    edgecolor='black',
    linewidth=1
)

key_countries = ['Austria', 'Germany', 'Belgium', 'Spain', 'Italy', 'Poland', 'Romania', 'Bulgaria']
for _, row in df_clean_regression.iterrows():
    if row['country_name'] in key_countries:
        plt.annotate(
            row['country_name'],
            xy=(row['gdp_per_capita'], row['recycling_rate']),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=9,
            weight='bold'
        )

plt.title("Country Clusters: Wealth vs. Recycling Performance", fontsize=14, fontweight='bold')
plt.xlabel("GDP per Capita (€)")
plt.ylabel("General Recycling Rate (%)")
plt.grid(alpha=0.3)
plt.legend(title='Cluster', loc='best')
plt.tight_layout()
plt.savefig("./figures/country_clusters.png", dpi=300, bbox_inches='tight')
plt.show()

print("\nCluster Profiles:")
print(df_clean_regression.groupby('cluster')[features_cluster].mean())

The clustering reveals that economic power is not the sole determinant of success. Cluster 1 combines high GDP (€53.5k) with strong governance (Index: 1.56) to maximize recycling (~52%). On the other hand, Cluster 0 and Cluster 2 share identical economic baselines (~€22k GDP) yet exhibit a 26.6% performance gap (41.3% vs. 14.7%).

This disparity tracks with Government Effectiveness (0.82 vs. 0.44), indicating that for middle-income nations, policy enforcement is a more critical driver than raw capital.

### Multivariate Regression (OLS)

Comparative OLS analysis quantifies driver impact. Two specifications are tested to resolve **multicollinearity**:

1.  **Full Model (10 vars):** High variance inflation; prone to overfitting.
2.  **Refined Model (5 vars):** Parsimonious selection representing distinct structural pillars:
    * *Economic:* **Private Investment** (Capital flow).
    * *Institutional:* **Government Effectiveness** (Enforcement capacity) and **Total Environment Tax** (Incentives by the government).
    * *Social:* **Education** (Compliance proxy).

**Visualization:** Standardized coefficients ($\beta$) allow direct magnitude comparison across predictors with different units.

In [ ]:
target = 'recycling_rate'
predictors_full = policy_drivers + context_vars
predictors_refined = [
    'priv_inv_per_capita',
    'total_environm_tax_per_capita',
    'government_effectiveness_estimate',
    'highschool_completed_pct'
]

X_full = df_clean_regression[predictors_full].copy()
y = df_clean_regression[target].copy()

X_full_std = (X_full - X_full.mean()) / X_full.std()
X_full_std = sm.add_constant(X_full_std)

model_full = sm.OLS(y, X_full_std).fit()

X_ref = df_clean_regression[predictors_refined].copy()
X_ref_std = (X_ref - X_ref.mean()) / X_ref.std()
X_ref_std = sm.add_constant(X_ref_std)

model_refined = sm.OLS(y, X_ref_std).fit()

print(f"Full model (10 vars)    -> R-squared Adj: {model_full.rsquared_adj:.3f} | AIC: {model_full.aic:.1f}")
print(f"Refined model (4 vars) -> R-squared Adj: {model_refined.rsquared_adj:.3f} | AIC: {model_refined.aic:.1f}")
print("\nWinning model: Refined model")
print(model_refined.summary())

coefs = model_refined.params.drop('const').sort_values()

fig, ax = plt.subplots(figsize=(10, 5))
colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in coefs.values]

coefs.plot(kind='barh', ax=ax, color=colors, edgecolor='black', width=0.7)

ax.set_title("Key Drivers of Recycling Rate\n(Refined Model - Standardized Coefficients)", fontsize=14, fontweight='bold')
ax.set_xlabel("Effect Size (Std. Deviations increase in Recycling Rate per 1 SD in predictor)")
ax.axvline(0, color='black', linewidth=1, linestyle='--')

for index, value in enumerate(coefs):
    offset = 0.1 if value > 0 else -0.5
    ax.text(value + offset, index, f"{value:.2f}", va='center', fontweight='bold', color='#333333')

ax.grid(alpha=0.3, axis='x')
plt.tight_layout()

plt.savefig("./figures/refined_model_drivers.png", dpi=300, bbox_inches='tight')
plt.show()

The refined model (4 vars) reduces noise, improving Adjusted  (0.40 vs. 0.37) compared to the full specification. 

The key driver for recycling is **Private Investment** ($\beta=7.49$) and is the only statistically significant predictor ($p=0.05$), identifying capital allocation as the primary lever for recycling performance. Education contributes marginally ($\beta=4.56$); conversely, environmental taxes and government effectiveness show no independent statistical significance in this model.

### Bivariate Analysis: Fiscal Policy

Pearson correlation ($r$) assesses the relationship between taxation subtypes and recycling performance.

**Distinction:**
* **Total Environmental Tax:** Broad fiscal policy/infrastructure funding.
* **Pollution Tax:** Specific levies on emissions. often **reactive** measures in high-pollution contexts rather than preventative drivers.

In [ ]:
target = 'recycling_rate'
tax_vars = ['total_environm_tax_per_capita', 'pollut_environm_tax_per_capita']
df_tax = df_all[['country_name', 'year', target] + tax_vars].copy()

df_tax[tax_vars + [target]] = df_tax.groupby('country_name')[tax_vars + [target]].transform(
    lambda x: x.interpolate(limit_direction='both')
)
df_tax = df_tax.dropna()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

sns.regplot(x='total_environm_tax_per_capita', y='recycling_rate', data=df_tax, 
            ax=ax1, scatter_kws={'alpha':0.4, 'color':'tab:blue'}, line_kws={'color':'red'})

r_tot, p_tot = pearsonr(df_tax['total_environm_tax_per_capita'], df_tax['recycling_rate'])
ax1.set_title(f'Total Environmental Tax Impact\nR = {r_tot:.2f} (p={p_tot:.3f})', fontsize=12, fontweight='bold')
ax1.set_xlabel('Total Environmental Tax (€ per capita)')
ax1.set_ylabel('Recycling Rate (%)')
ax1.grid(alpha=0.3)

sns.regplot(x='pollut_environm_tax_per_capita', y='recycling_rate', data=df_tax, 
            ax=ax2, scatter_kws={'alpha':0.4, 'color':'tab:green'}, line_kws={'color':'darkgreen'})

r_pol, p_pol = pearsonr(df_tax['pollut_environm_tax_per_capita'], df_tax['recycling_rate'])
ax2.set_title(f'Pollution Tax Impact\nR = {r_pol:.2f} (p={p_pol:.3f})', fontsize=12, fontweight='bold')
ax2.set_xlabel('Pollution Environmental Tax (€ per capita)')
ax2.set_ylabel('')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("./figures/tax_correlation_comparison.png", dpi=300)
plt.show()

Data reveals that direct pollution taxes comprise a minor share of fiscal policy. High performers like Germany maintain low direct levies despite superior recycling outcomes.
The weak positive association $(R=0.26)$ contrasts with broader environmental taxation $(R=0.68)$, indicating that pollution-specific levies are currently under-leveraged compared to general infrastructure funding.

As a policy recommendation, there is significant scope to implement targeted pollution taxes. Shifting fiscal pressure from general revenue to specific behavioral penalties represents an unexploited mechanism to further accelerate circular transition. The green line shows the correlation for those countries that do implement said taxes.

# Dataset preparation for recycling rate prediction

In [ ]:
countries_to_drop = [
    "Bosnia and Herzegovina",
    "Montenegro",
    "North Macedonia",
    "Türkiye",
    "Serbia",
    "Albania"
]

res_d1_d3 = res_d1_d3[
    ~res_d1_d3["country_name"].isin(countries_to_drop)
].copy()

display(res_d1_d3)
sorted(res_d1_d3["country_name"].unique())


In [ ]:
def build_horizon_delta_dataset(
    df,
    horizon=7,
    start_year=2000,
    end_year=None,
    target_col="recycling_rate_filled",
    feature_cols=None
):
    """
    Builds a dataset where the target is the change in target_col over `horizon` years.
    """

    if end_year is None:
        end_year = df["year"].max() - horizon

    # anchor data (t)
    df_t = df[
        (df["year"] >= start_year) &
        (df["year"] <= end_year)
    ][["country_name", "year"] + feature_cols].copy()

    # future data (t + horizon)
    df_t_future = df[
        (df["year"] >= start_year + horizon) &
        (df["year"] <= end_year + horizon)
    ][["country_name", "year", target_col]].copy()

    df_t_future["year"] -= horizon

    # merge
    merged = df_t.merge(
        df_t_future,
        on=["country_name", "year"],
        how="inner",
        suffixes=("", "_future")
    )

    # delta target
    merged[f"delta_{target_col}_{horizon}yr"] = (
        merged[f"{target_col}_future"] - merged[target_col]
    )

    return merged


In [ ]:
feature_cols = [
    "recycling_rate_filled",
    "gdp_per_capita_filled",
    "urban_population_pct_filled",
    "internet_users_pct_filled",
    "renewable_energy_pct_filled",
    "tourism_arrivals_filled",
    "population_total_filled",
    "manufacturing_value_added_pct_gdp_filled",
    "government_effectiveness_estimate_filled",
    "highschool_completed_pct_filled",
    "household_exp_percapita_filled"
]

# build dataset with horizon recycling rates (in 7 years)
df_7yr = build_horizon_delta_dataset(
    feature_cols = feature_cols,
    df=res_d1_d3,
    horizon=7
)

display(df_7yr)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_7yr["delta_recycling_rate_filled_7yr"], bins=30)
plt.xlabel("7-year change in recycling rate")
plt.ylabel("Frequency")
plt.title("Distribution of 7-year recycling rate changes")
plt.show()

## Check for diminsihing returns 

We want to check that higher starting rate → smaller improvement

In [ ]:
plt.scatter(
    df_7yr["recycling_rate_filled"],
    df_7yr["delta_recycling_rate_filled_7yr"],
    alpha=0.5
)
plt.xlabel("Initial recycling rate")
plt.ylabel("7-year change")
plt.title("Diminishing returns check")
plt.show()

## Split the data for delta = 7

In [ ]:
# Create training and validation/test datasets
train_df = df_7yr[df_7yr["year"] <= 2012].copy()
val_df   = df_7yr[df_7yr["year"] > 2012].copy()

In [ ]:
# Defining feature and target variables
target = "delta_recycling_rate_filled_7yr"

X_train = train_df[feature_cols]
y_train = train_df[target]

X_val = val_df[feature_cols]
y_val = val_df[target]

## Baseline Model: Linear regression (with regularization)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

# Build a regression pipeline that scales features and fits a Ridge regression model
baseline_model = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=1.0))
])

baseline_model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# Generate predictions on validation set and evaluate model performance using MAE and R²
y_pred = baseline_model.predict(X_val)

print("MAE:", mean_absolute_error(y_val, y_pred))
print("R²:", r2_score(y_val, y_pred))

In [ ]:
# Check coefficients of the regression model
coefs = pd.Series(
    baseline_model.named_steps["model"].coef_,
    index=X_train.columns
).sort_values()

coefs

In [ ]:
# Plot predicted over actual 7-year change
plt.scatter(y_val, y_pred, alpha=0.6)
plt.plot([y_val.min(), y_val.max()],
         [y_val.min(), y_val.max()],
         "--", color="black")
plt.xlabel("Actual 7-year change")
plt.ylabel("Predicted 7-year change")
plt.title("Validation: Predicted vs Actual")
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)

# Define helper functions
def evaluate_model(model, X_train, y_train, X_val, y_val, name="model"):
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    mae = mean_absolute_error(y_val, preds)
    r2 = r2_score(y_val, preds)
    return mae, r2, model, preds

def cast_int_or_none(x):
    return None if pd.isna(x) else int(x)

## Non-linear Model: Random Forest

In [ ]:
# Define basline RF model 
rf_model = RandomForestRegressor(n_estimators=500, random_state=42)

rf_mae, rf_r2, _, _ = evaluate_model(rf_model, X_train, y_train, X_val, y_val)

print("RF Baseline MAE:", rf_mae)
print("RF Baseline R²:", rf_r2)

In [ ]:
# Manual tuning to optimize hyperparameters (n_estimators, max_depth, min_samples_leaf, min_samples_split, max_features)
results = []

for n_estimators in [200, 400, 800]:
    for max_depth in [None, 8, 12]:
        for min_leaf in [1, 2, 4]:
            for min_split in [2, 5, 10]:
                for max_feat in ["sqrt", "log2", None]:

                    rf = RandomForestRegressor(
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        min_samples_leaf=min_leaf,
                        min_samples_split=min_split,
                        max_features=max_feat,
                        random_state=42,
                    )

                    mae, r2, _, _ = evaluate_model(rf, X_train, y_train, X_val, y_val)

                    results.append({
                        "n_estimators": n_estimators,
                        "max_depth": max_depth,
                        "min_samples_leaf": min_leaf,
                        "min_samples_split": min_split,
                        "max_features": max_feat,
                        "MAE": mae,
                        "R2": r2
                    })

rf_results = pd.DataFrame(results).sort_values("MAE")

rf_results

In [ ]:
# Investigate best performing RF model (MAE, R2 & Importances)
best_rf = RandomForestRegressor(
    n_estimators=rf_results.iloc[0]["n_estimators"],
    max_depth=cast_int_or_none(rf_results.iloc[0]["max_depth"]),
    min_samples_leaf=rf_results.iloc[0]["min_samples_leaf"],
    min_samples_split=rf_results.iloc[0]["min_samples_split"],
    max_features=rf_results.iloc[0]["max_features"],
    random_state=42,
)

best_rf_mae, best_rf_r2, best_rf, best_rf_preds = evaluate_model(best_rf, X_train, y_train, X_val, y_val)

print("Best RF MAE:", best_rf_mae)
print("Best RF R²:", best_rf_r2)

plt.scatter(y_val, best_rf_preds, alpha=0.6)
plt.plot([y_val.min(), y_val.max()],
         [y_val.min(), y_val.max()],
         "--", color="black")
plt.xlabel("Actual 7-year change")
plt.ylabel("Predicted 7-year change")
plt.title("Validation: Predicted vs Actual")
plt.show()

importances = pd.Series(best_rf.feature_importances_, index=feature_cols).sort_values(ascending=False)
print(importances)

## Non-linear Model: Gradient Boosting

In [ ]:
# Define baseline Gradient Boosting model
gbr = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    random_state=42
)

gbr_mae, gbr_r2, _, _ = evaluate_model(gbr, X_train, y_train, X_val, y_val)

print("GBR Baseline MAE:", gbr_mae)
print("GBR Baseline R²:", gbr_r2)

In [ ]:
# Manual tuning to optimize hyperparameters (n_estimators, learning_rate, max_depth, min_samples_leaf, subsample)

gbr_results = []

for n_estimators in [200, 400, 800]:
    for learning_rate in [0.03, 0.05, 0.1]:
        for max_depth in [2, 3, 4]:
            for min_leaf in [1, 3, 5]:
                for subsample in [0.7, 0.9, 1.0]:

                    gbr = GradientBoostingRegressor(
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth,
                        min_samples_leaf=min_leaf,
                        subsample=subsample,
                        random_state=42
                    )
                    
                    mae, r2, _, _ = evaluate_model(gbr, X_train, y_train, X_val, y_val)

                    gbr_results.append({
                        "n_estimators": n_estimators,
                        "learning_rate": learning_rate,
                        "max_depth": max_depth,
                        "min_samples_leaf": min_leaf,
                        "subsample": subsample,
                        "MAE": mae,
                        "R2": r2
                    })

gbr_results_df = (
    pd.DataFrame(gbr_results)
    .sort_values("MAE")
    .reset_index(drop=True)
)

gbr_results_df

In [ ]:
# Investigate best performing GBR model (MAE, R2 & Importances)

best_gbr = GradientBoostingRegressor(
    n_estimators=cast_int_or_none(gbr_results_df.iloc[0]["n_estimators"]),
    learning_rate=gbr_results_df.iloc[0]["learning_rate"],
    max_depth=cast_int_or_none(gbr_results_df.iloc[0]["max_depth"]),
    min_samples_leaf=cast_int_or_none(gbr_results_df.iloc[0]["min_samples_leaf"]),
    subsample=gbr_results_df.iloc[0]["subsample"],
    random_state=42,
)

best_gbr_mae, best_gbr_r2, best_gbr, best_gbr_preds = evaluate_model(best_gbr, X_train, y_train, X_val, y_val)

print("Best GBR MAE:", best_gbr_mae)
print("Best GBR R²:", best_gbr_r2)

plt.scatter(y_val, best_gbr_preds, alpha=0.6)
plt.plot([y_val.min(), y_val.max()],
         [y_val.min(), y_val.max()],
         "--", color="black")
plt.xlabel("Actual 7-year change")
plt.ylabel("Predicted 7-year change")
plt.title("Validation: Predicted vs Actual")
plt.show()

importances = pd.Series(best_gbr.feature_importances_, index=feature_cols).sort_values(ascending=False)
print(importances)

## Non-linear Model: Histogram-based Gradient Boosting

In [ ]:
# Define basemodel HGB model

hgb = HistGradientBoostingRegressor(
    loss="absolute_error",
    max_iter=300,
    learning_rate=0.05,
    random_state=42
)

hgb_mae, hgb_r2, _, _ = evaluate_model(hgb, X_train, y_train, X_val, y_val)


print("HGB Baseline MAE:", hgb_mae)
print("HGB Baseline R²:", hgb_r2)

In [ ]:
# Manual tuning to optimize hyperparameters (max_iter, learning_rate, max_depth, min_samples_leaf)

hgb_results = []

for max_iter in [300, 600, 1000]:
    for learning_rate in [0.03, 0.05, 0.1]:
        for max_depth in [3, 5, 7]:
            for min_samples_leaf in [10, 20, 40]:

                hgb = HistGradientBoostingRegressor(
                    max_iter=max_iter,
                    learning_rate=learning_rate,
                    max_depth=max_depth,
                    min_samples_leaf=min_samples_leaf,
                    random_state=42
                )

                mae, r2, _, _ = evaluate_model(hgb, X_train, y_train, X_val, y_val)

                hgb_results.append({
                    "max_iter": max_iter,
                    "learning_rate": learning_rate,
                    "max_depth": max_depth,
                    "min_samples_leaf": min_samples_leaf,
                    "MAE": mae,
                    "R2": r2
                })

hgb_results_df = (
    pd.DataFrame(hgb_results)
    .sort_values("MAE")
    .reset_index(drop=True)
)

hgb_results_df


In [ ]:
# Investigate best performing HGB model (MAE, R2)

best_hgb = HistGradientBoostingRegressor(
    max_iter=cast_int_or_none(hgb_results_df.iloc[0]["max_iter"]),
    learning_rate=hgb_results_df.iloc[0]["learning_rate"],
    max_depth=cast_int_or_none(hgb_results_df.iloc[0]["max_depth"]),
    min_samples_leaf=cast_int_or_none(hgb_results_df.iloc[0]["min_samples_leaf"]),
    random_state=42,
)

best_hgb_mae, best_hgb_r2, best_hgb, best_hgb_preds = evaluate_model(best_hgb, X_train, y_train, X_val, y_val)

print("Best HGB MAE:", best_hgb_mae)
print("Best hgb R²:", best_hgb_r2)

plt.scatter(y_val, best_hgb_preds, alpha=0.6)
plt.plot([y_val.min(), y_val.max()],
         [y_val.min(), y_val.max()],
         "--", color="black")
plt.xlabel("Actual 7-year change")
plt.ylabel("Predicted 7-year change")
plt.title("Validation: Predicted vs Actual")
plt.show()

## Non-linear Model: Extreme Gradient Boosting

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBRegressor

# Define Baseline XGB model
xgb = XGBRegressor(
    objective="reg:absoluteerror", 
    n_estimators=400,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_mae, xgb_r2, _, _ = evaluate_model(xgb, X_train, y_train, X_val, y_val)

print("XGB Baseline MAE:", xgb_mae)
print("XGB Baseline R²:", xgb_r2)

In [ ]:
from xgboost import XGBRegressor
import pandas as pd

# Manual tuning to optimize hyperparameters (n_estimators, learning_rate, max_depth, subsample, colsample_bytree)

xgb_results = []

for n_estimators in [300, 600, 1000]:
    for learning_rate in [0.03, 0.05, 0.1]:
        for max_depth in [3, 4, 5]:
            for subsample in [0.7, 0.9, 1.0]:
                for colsample in [0.7, 0.9, 1.0]:

                    xgb = XGBRegressor(
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth,
                        subsample=subsample,
                        colsample_bytree=colsample,
                        random_state=42,
                    )

                    mae, r2, _, _ = evaluate_model(xgb, X_train, y_train, X_val, y_val)


                    xgb_results.append({
                        "n_estimators": n_estimators,
                        "learning_rate": learning_rate,
                        "max_depth": max_depth,
                        "subsample": subsample,
                        "colsample_bytree": colsample,
                        "MAE": mae,
                        "R2": r2
                    })

xgb_results_df = (
    pd.DataFrame(xgb_results)
    .sort_values("MAE")
    .reset_index(drop=True)
)

xgb_results_df

In [ ]:
# Investigate best performing XGB model (MAE, R2 & Importances)

best_xgb = XGBRegressor(
    n_estimators=cast_int_or_none(xgb_results_df.iloc[0]["n_estimators"]),
    learning_rate=xgb_results_df.iloc[0]["learning_rate"],
    max_depth=cast_int_or_none(xgb_results_df.iloc[0]["max_depth"]),
    subsample=xgb_results_df.iloc[0]["subsample"],
    colsample_bytree=xgb_results_df.iloc[0]["colsample_bytree"],
    random_state=42,
)

best_xgb_mae, best_xgb_r2, best_xgb, best_xgb_preds = evaluate_model(best_xgb, X_train, y_train, X_val, y_val)

print("Best XGB MAE:", best_xgb_mae)
print("Best XGB R²:", best_xgb_r2)

plt.scatter(y_val, best_xgb_preds, alpha=0.6)
plt.plot([y_val.min(), y_val.max()],
         [y_val.min(), y_val.max()],
         "--", color="black")
plt.xlabel("Actual 7-yeaxr change")
plt.ylabel("Predicted 7-year change")
plt.title("Validation: Predicted vs Actual")
plt.show()

importances = pd.Series(best_xgb.feature_importances_, index=feature_cols).sort_values(ascending=False)
print(importances)

## Would we see better MAE with 5 years delta (instead of 7)? 

In [ ]:
# Resplitting dataset for delta of 5 years

df_5yr = build_horizon_delta_dataset(
    feature_cols=feature_cols,
    df=res_d1_d3,
    horizon=5
)

train_df_5 = df_5yr[df_5yr["year"] <= 2014]
val_df_5   = df_5yr[df_5yr["year"] > 2014]

target_5 = "delta_recycling_rate_filled_5yr"

X_train_5 = train_df_5[feature_cols]
y_train_5 = train_df_5[target_5]

X_val_5 = val_df_5[feature_cols]
y_val_5 = val_df_5[target_5]

# Evaluatign previous optimized models for delta of 5 years
best_rf_5_mae , best_rf_5_r2, _, _ = evaluate_model(best_rf,  X_train_5, y_train_5, X_val_5, y_val_5, "RF (5yr)")
best_gbr_5_mae , best_gbr_5_r2, _, _ =evaluate_model(best_gbr, X_train_5, y_train_5, X_val_5, y_val_5, "GBR (5yr)")
best_hgb_5_mae , best_hgb_5_r2, _, _ =evaluate_model(best_hgb, X_train_5, y_train_5, X_val_5, y_val_5, "HGB (5yr)")
best_xgb_5_mae , best_xgb_5_r2, _, _ =evaluate_model(best_xgb, X_train_5, y_train_5, X_val_5, y_val_5, "XGB (5yr)")

results_5yr = {
    "RF (5yr)": (best_rf_5_mae, best_rf_5_r2),
    "GBR (5yr)": (best_gbr_5_mae, best_gbr_5_r2),
    "HGB (5yr)": (best_hgb_5_mae, best_hgb_5_r2),
    "XGB (5yr)": (best_xgb_5_mae, best_xgb_5_r2),
}

for model, (mae, r2) in results_5yr.items():
    print(f"{model:10s} | MAE: {mae:.3f} | R²: {r2:.3f}")

Answer is no but we might need to retune the models for 5 years delta

# Predict recycling rates for dataset in 2030

In [ ]:
predictions__df = res_d1_d3[res_d1_d3["year"] == res_d1_d3["year"].max()].copy()

# Features for prediction
X_latest = predictions__df[feature_cols]

# Predict 7-year delta from latest year
predicted_delta = best_xgb.predict(X_latest)

# Compute projected 2030 rate
predictions__df["predicted_delta_2030"] = predicted_delta
predictions__df["predicted_rate_2030"] = predictions__df["recycling_rate_filled"] + predicted_delta

eu_target = 60 
predictions__df["meets_target"] = predictions__df["predicted_rate_2030"] >= eu_target

projection_2030 = predictions__df[
    ["country_name", "recycling_rate_filled", "predicted_delta_2030", "predicted_rate_2030", "meets_target"]
].sort_values("predicted_rate_2030", ascending=False).reset_index(drop=True)

projection_2030

In [ ]:
# show predicted deltas by 2030
plt.figure(figsize=(10,6))
sns.histplot(projection_2030["predicted_delta_2030"], bins=7, kde=True, color='skyblue')
plt.axvline(0, color='red', linestyle='--')
plt.xlabel("Predicted 7-year increase in recycling rate")
plt.ylabel("Number of countries")
plt.title("Distribution of Predicted Recycling Rate Increase by 2030")
plt.show()

In [ ]:
# Show predicted final recycling rates by 2030

plt.figure(figsize=(10,6))
sns.histplot(projection_2030["predicted_rate_2030"], bins=9, kde=True, color='green')
plt.axvline(eu_target, color='red', linestyle='--', label=f'EU Target ({eu_target}%)')
plt.xlabel("Predicted recycling rate in 2030 (%)")
plt.ylabel("Number of countries")
plt.title("Distribution of Predicted Recycling Rates in 2030")
plt.legend()
plt.show()

In [ ]:
# Show Current vs preidcted recycling rates by 2030

plt.figure(figsize=(10,6))
sns.scatterplot(
    x="recycling_rate_filled",
    y="predicted_rate_2030",
    hue="meets_target",
    data=projection_2030,
    palette={True: 'green', False: 'red'},
    s=100
)
plt.plot([0, 100], [0, 100], 'k--', alpha=0.7)  # diagonal
plt.xlabel("Current recycling rate (%)")
plt.ylabel("Predicted rate in 2030 (%)")
plt.title("Current vs Predicted Recycling Rates")
plt.show()

## Countries with biggest predicted improvements

In [ ]:
# Show countries with biggets predicted improvements 
top_increases = projection_2030.sort_values("predicted_delta_2030", ascending=False).head(10)
top_increases[["country_name", "recycling_rate_filled", "predicted_delta_2030", "predicted_rate_2030"]]

## Countries by outcome analysis

In [ ]:
# Show most contriubting variables to recycling rates deltas
importances = best_xgb.feature_importances_
feat_imp_df = pd.DataFrame({
    "feature": feature_cols,
    "importance": importances
}).sort_values("importance", ascending=False)

# Plot
plt.figure(figsize=(10,6))
sns.barplot(x="importance", y="feature", data=feat_imp_df, palette="viridis")
plt.title("Global Feature Importance (XGBoost)")
plt.show()

In [ ]:
!pip install shap

In [ ]:
import shap

explainer = shap.Explainer(best_xgb, X_latest)
shap_values = explainer(X_latest)

# Global summary
shap.summary_plot(shap_values, X_latest, feature_names=feature_cols)

# Mean absolute SHAP value per feature
shap_df = pd.DataFrame({
    "feature": feature_cols,
    "mean_abs_shap": np.abs(shap_values.values).mean(axis=0)
}).sort_values("mean_abs_shap", ascending=False)

print(shap_df)



In [ ]:
import math
import matplotlib.pyplot as plt
import seaborn as sns

# Split dataset into achievers and non-achievers
achievers = predictions__df[predictions__df["meets_target"]].copy()
non_achievers = predictions__df[~predictions__df["meets_target"]].copy()

# Determine grid size
n_features = len(feature_cols)
n_cols = 3 
n_rows = math.ceil(n_features / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*4))
axes = axes.flatten()  # flatten for easy indexing

for i, feature in enumerate(feature_cols):
    sns.kdeplot(achievers[feature], label="Achievers", fill=True, ax=axes[i])
    sns.kdeplot(non_achievers[feature], label="Non-Achievers", fill=True, ax=axes[i])
    axes[i].set_title(f"{feature} distribution")
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel("Density")
    axes[i].legend()

# Remove empty subplots if any
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()
